# 2. Acquire the Data


## Finding Data Sources

There are three place to get onion price and quantity information by market. 

1. **[Agmarket](http://agmarknet.nic.in/)** - This is the website run by the Directorate of Marketing & Inspection (DMI), Ministry of Agriculture, Government of India and provides daily price and arrival data for all agricultural commodities at national and state level. Unfortunately, the link to get Market-wise Daily Report for Specific Commodity (Onion for us) leads to a multipage aspx entry form to get data for each date. So it is like to require an involved scraper to get the data. Too much effort - Move on. Here is the best link to go to get what is available - http://agmarknet.nic.in/agnew/NationalBEnglish/SpecificCommodityWeeklyReport.aspx?ss=1


2. **[Data.gov.in](https://data.gov.in/)** - This is normally a good place to get government data in a machine readable form like csv or xml. The Variety-wise Daily Market Prices Data of Onion is available for each year as an XML but unfortunately it does not include quantity information that is needed. It would be good to have both price and quantity - so even though this is easy, lets see if we can get both from a different source. Here is the best link to go to get what is available - https://data.gov.in/catalog/variety-wise-daily-market-prices-data-onion#web_catalog_tabs_block_10


3. **[NHRDF](http://nhrdf.org/en-us/)** - This is the website of National Horticultural Research & Development Foundation and maintains a database on Market Arrivals and Price, Area and Production and Export Data for three commodities - Garlic, Onion and Potatoes. We are in luck! It also has data from 1996 onwards and has only got one form to fill to get the data in a tabular form. Further it also has production and export data. Excellent. Lets use this. Here is the best link to got to get all that is available - http://nhrdf.org/en-us/DatabaseReports


## Scraping the Data


### Ways to Scrape Data
Now we can do this in two different levels of sophistication

1. **Automate the form filling process**: The form on this page looks simple. But viewing source in the browser shows there form to fill with hidden fields and we will need to access it as a browser to get the session fields and then submit the form. This is a little bit more complicated than simple scraping a table on a webpage

2. **Manually fill the form**: What if we manually fill the form with the desired form fields and then save the page as a html file. Then we can read this file and just scrape the table from it. Lets go with the simple way for now.


### Scraping - Manual Form Filling

So let us fill the form to get a small subset of data and test our scraping process.  We will start by getting the [Monthwise Market Arrivals](http://nhrdf.org/en-us/MonthWiseMarketArrivals). 

- Crop Name: Onion
- Month: January
- Market: All
- Year: 2016

The saved webpage is available at [MonthWiseMarketArrivalsJan2016.html](MonthWiseMarketArrivalsJan2016.html)

### Understand the HTML Structure

We need to scrape data from this html page... So let us try to understand the structure of the page.

1. You can view the source of the page - typically Right Click and View Source on any browser and that would give your the source HTML for any page.

2. You can open the developer tools in your browser and investigate the structure as you mouse over the page 

3. We can use a tools like [Selector Gadget](http://selectorgadget.com/) to understand the id's and classes' used in the web page

Our data is under the **&lt;table&gt;** tag 

### Exercise #1

Find the number of tables in the HTML Structure of [MonthWiseMarketArrivalsJan2016.html](MonthWiseMarketArrivalsJan2016.html)?

### Find all the Tables 

In [1]:
# Import the library we need, which is Pandas
import pandas as pd

In [2]:
# Read all the tables from the html document 
AllTables = pd.read_html('MonthWiseMarketArrivals2017.htm')

In [3]:
# Let us find out how many tables has it found?
len(AllTables)

5

In [4]:
type(AllTables)

list

### Exercise #2
Find the exact table of data we want in the list of AllTables?

### Get the exact table
To read the exact table we need to pass in an identifier value which would identify the table. We can use the `attrs` parameter in read_html to do so. The parameter we will pass is the `id` variable

In [5]:
# So can we read our exact table
OneTable = pd.read_html('MonthWiseMarketArrivals2017.htm', 
                      attrs = {'id' : 'dnn_ctr974_MonthWiseMarketArrivals_GridView1'})

In [6]:
# So how many tables have we got now
len(OneTable)

1

In [7]:
# Show the table of data identifed by pandas with just the first five rows
OneTable[0].head()

,0,1,2,3,4,5,6
0,Market,Month Name,Year,Arrival (q),Price Minimum (Rs/q),Price Maximum (Rs/q),Modal Price (Rs/q)
1,ABOHAR(PB),January,2017,200,750,1000,850
2,AGRA(UP),January,2017,69360,730,871,815
3,AGRA(UP),February,2017,58850,767,918,872
4,AGRA(UP),March,2017,76370,790,910,855


However, we have not got the header correctly in our dataframe. Let us see if we can fix this.

To get help on any function just use `??` before the function to help. Run this function and see what additional parameter you need to define to get the header correctly

In [8]:
??pd.read_html

### Exercise #3
Read the html file again and ensure that the correct header is identifed by pandas?

In [9]:
OneTable = pd.read_html('MonthWiseMarketArrivalsJan2016.html', header= 0,
                      attrs = {'id' : 'dnn_ctr974_MonthWiseMarketArrivals_GridView1'})

Show the top five rows of the dataframe you have read to ensure the headers are now correct.

In [10]:
OneTable[0].head()

,Market,Month Name,Year,Arrival (q),Price Minimum (Rs/q),Price Maximum (Rs/q),Modal Price (Rs/q)
0,AGRA(UP),January,2016,134200,1039,1443,1349
1,AHMEDABAD(GUJ),January,2016,198390,646,1224,997
2,AHMEDNAGAR(MS),January,2016,208751,175,1722,1138
3,AJMER(RAJ),January,2016,4247,722,1067,939
4,ALIGARH(UP),January,2016,12350,1219,1298,1257


###  Dataframe Viewing  

In [11]:
# Let us store the dataframe in a df variable. You will see that as a very common convention in data science pandas use
df = OneTable[0]

In [12]:
# Shape of the dateset - number of rows & number of columns in the dataframe
df.shape

(84, 7)

In [13]:
# Get the names of all the columns 
df.columns

Index(['Market', 'Month Name', 'Year', 'Arrival (q)', 'Price Minimum (Rs/q)',
       'Price Maximum (Rs/q)', 'Modal Price (Rs/q)'],
      dtype='object')

In [14]:
df.index

RangeIndex(start=0, stop=84, step=1)

In [15]:
# Can we see sample rows - the top 5 rows
df.head(10)

,Market,Month Name,Year,Arrival (q),Price Minimum (Rs/q),Price Maximum (Rs/q),Modal Price (Rs/q)
0,AGRA(UP),January,2016,134200,1039,1443,1349
1,AHMEDABAD(GUJ),January,2016,198390,646,1224,997
2,AHMEDNAGAR(MS),January,2016,208751,175,1722,1138
3,AJMER(RAJ),January,2016,4247,722,1067,939
4,ALIGARH(UP),January,2016,12350,1219,1298,1257
5,ALWAR(RAJ),January,2016,9788,625,1200,912
6,AMRITSAR(PB),January,2016,24800,913,1308,1160
7,BALLIA(UP),January,2016,600,1400,1500,1460
8,BANGALORE,January,2016,507223,200,1943,1448
9,BAREILLY(UP),January,2016,18435,1149,1149,1149


In [16]:
# Can we see sample rows - the bottom 5 rows
df.tail()

,Market,Month Name,Year,Arrival (q),Price Minimum (Rs/q),Price Maximum (Rs/q),Modal Price (Rs/q)
79,UDAIPUR(RAJ),January,2016,6456,386,1307,846
80,VANI(MS),January,2016,60983,767,1323,1007
81,VARANASI(UP),January,2016,28900,1460,1503,1484
82,YEOLA(MS),January,2016,437432,437,1272,1034
83,NaN,NaN,Total,9307923,751(Avg),1490(Avg),1186(Avg)


In [17]:
# Can we access a specific columns
df["Market"].head()

0          AGRA(UP)
1    AHMEDABAD(GUJ)
2    AHMEDNAGAR(MS)
3        AJMER(RAJ)
4       ALIGARH(UP)
Name: Market, dtype: object

In [18]:
# Using the dot notation
df.Market.head()

0          AGRA(UP)
1    AHMEDABAD(GUJ)
2    AHMEDNAGAR(MS)
3        AJMER(RAJ)
4       ALIGARH(UP)
Name: Market, dtype: object

In [19]:
df.head()

,Market,Month Name,Year,Arrival (q),Price Minimum (Rs/q),Price Maximum (Rs/q),Modal Price (Rs/q)
0,AGRA(UP),January,2016,134200,1039,1443,1349
1,AHMEDABAD(GUJ),January,2016,198390,646,1224,997
2,AHMEDNAGAR(MS),January,2016,208751,175,1722,1138
3,AJMER(RAJ),January,2016,4247,722,1067,939
4,ALIGARH(UP),January,2016,12350,1219,1298,1257


In [21]:
# Selecting specific column and rows
df[0:5][["Market", "Year"]]

,Market,Year
0,AGRA(UP),2016
1,AHMEDABAD(GUJ),2016
2,AHMEDNAGAR(MS),2016
3,AJMER(RAJ),2016
4,ALIGARH(UP),2016


Accessing Data
- By direct labels
- By location - index
- By location - labels

In [22]:
df["Market"].head()

0          AGRA(UP)
1    AHMEDABAD(GUJ)
2    AHMEDNAGAR(MS)
3        AJMER(RAJ)
4       ALIGARH(UP)
Name: Market, dtype: object

In [23]:
df.head()

,Market,Month Name,Year,Arrival (q),Price Minimum (Rs/q),Price Maximum (Rs/q),Modal Price (Rs/q)
0,AGRA(UP),January,2016,134200,1039,1443,1349
1,AHMEDABAD(GUJ),January,2016,198390,646,1224,997
2,AHMEDNAGAR(MS),January,2016,208751,175,1722,1138
3,AJMER(RAJ),January,2016,4247,722,1067,939
4,ALIGARH(UP),January,2016,12350,1219,1298,1257


In [24]:
df.loc[0:5,"Year"]

0    2016
1    2016
2    2016
3    2016
4    2016
5    2016
Name: Year, dtype: object

In [25]:
df.iloc[0:4,0:3].head(1)

,Market,Month Name,Year
0,AGRA(UP),January,2016


In [26]:
df.Year.unique()

array(['2016', 'Total'], dtype=object)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
Market                  83 non-null object
Month Name              83 non-null object
Year                    84 non-null object
Arrival (q)             84 non-null int64
Price Minimum (Rs/q)    84 non-null object
Price Maximum (Rs/q)    84 non-null object
Modal Price (Rs/q)      84 non-null object
dtypes: int64(1), object(6)
memory usage: 4.7+ KB


In [28]:
df.describe()

,Arrival (q)
count,8.400000e+01
mean,2.216172e+05
std,1.017445e+06
min,4.000000e+01
25%,1.140450e+04
50%,3.593000e+04
75%,1.379715e+05
max,9.307923e+06


In [29]:
x = 2

In [30]:
# Access 10 - 15 row, Year & Arrival columns
- iloc
- loc

NameError: name 'iloc' is not defined

In [31]:
df.iloc[10:15, 1:3 ]

,Month Name,Year
10,January,2016
11,January,2016
12,January,2016
13,January,2016
14,January,2016


In [32]:
# Works both ways
df["Market"][0:5]

0          AGRA(UP)
1    AHMEDABAD(GUJ)
2    AHMEDNAGAR(MS)
3        AJMER(RAJ)
4       ALIGARH(UP)
Name: Market, dtype: object

In [33]:
#Getting unique values of State
pd.unique(df['Market'])

array(['AGRA(UP)', 'AHMEDABAD(GUJ)', 'AHMEDNAGAR(MS)', 'AJMER(RAJ)',
       'ALIGARH(UP)', 'ALWAR(RAJ)', 'AMRITSAR(PB)', 'BALLIA(UP)',
       'BANGALORE', 'BAREILLY(UP)', 'BELGAUM(KNT)', 'BHATINDA(PB)',
       'BHAVNAGAR(GUJ)', 'BHUBNESWER(OR)', 'BIJAPUR(KNT)', 'BURDWAN(WB)',
       'CHAKAN(MS)', 'CHANDIGARH', 'CHANDVAD(MS)', 'CHENNAI', 'DEESA(GUJ)',
       'DEHRADOON(UTT)', 'DELHI', 'DEVALA(MS)', 'DHAVANGERE(KNT)',
       'DHULIA(MS)', 'GONDAL(GUJ)', 'GUWAHATI', 'HASSAN(KNT)',
       'HOSHIARPUR(PB)', 'HUBLI(KNT)', 'HYDERABAD', 'INDORE(MP)', 'JAIPUR',
       'JALANDHAR(PB)', 'JALGAON(MS)', 'JAMMU', 'JAMNAGAR(GUJ)',
       'JODHPUR(RAJ)', 'KALVAN(MS)', 'KANPUR(UP)', 'KARNAL(HR)',
       'KHANNA(PB)', 'KOLHAPUR(MS)', 'KOLKATA', 'KOPERGAON(MS)',
       'KOTA(RAJ)', 'KURNOOL(AP)', 'LASALGAON(MS)', 'LONAND(MS)',
       'LUCKNOW', 'MAHUVA(GUJ)', 'MALEGAON(MS)', 'MANMAD(MS)', 'MUMBAI',
       'NAGPUR', 'NEWASA(MS)', 'NIPHAD(MS)', 'PALAYAM(KER)', 'PATIALA(PB)',
       'PATNA', 'PHALTAN (MS)',

In [35]:
type(df)

pandas.core.frame.DataFrame

In [36]:
type(df.Market)

pandas.core.series.Series

In [37]:
df.dtypes

Market                  object
Month Name              object
Year                    object
Arrival (q)              int64
Price Minimum (Rs/q)    object
Price Maximum (Rs/q)    object
Modal Price (Rs/q)      object
dtype: object

In [38]:
import numpy as np

In [43]:
df.Market.values

array(['AGRA(UP)', 'AHMEDABAD(GUJ)', 'AHMEDNAGAR(MS)', 'AJMER(RAJ)',
       'ALIGARH(UP)', 'ALWAR(RAJ)', 'AMRITSAR(PB)', 'BALLIA(UP)',
       'BANGALORE', 'BAREILLY(UP)', 'BELGAUM(KNT)', 'BHATINDA(PB)',
       'BHAVNAGAR(GUJ)', 'BHUBNESWER(OR)', 'BIJAPUR(KNT)', 'BURDWAN(WB)',
       'CHAKAN(MS)', 'CHANDIGARH', 'CHANDVAD(MS)', 'CHENNAI', 'DEESA(GUJ)',
       'DEHRADOON(UTT)', 'DELHI', 'DEVALA(MS)', 'DHAVANGERE(KNT)',
       'DHULIA(MS)', 'GONDAL(GUJ)', 'GUWAHATI', 'HASSAN(KNT)',
       'HOSHIARPUR(PB)', 'HUBLI(KNT)', 'HYDERABAD', 'INDORE(MP)', 'JAIPUR',
       'JALANDHAR(PB)', 'JALGAON(MS)', 'JAMMU', 'JAMNAGAR(GUJ)',
       'JODHPUR(RAJ)', 'KALVAN(MS)', 'KANPUR(UP)', 'KARNAL(HR)',
       'KHANNA(PB)', 'KOLHAPUR(MS)', 'KOLKATA', 'KOPERGAON(MS)',
       'KOTA(RAJ)', 'KURNOOL(AP)', 'LASALGAON(MS)', 'LONAND(MS)',
       'LUCKNOW', 'MAHUVA(GUJ)', 'MALEGAON(MS)', 'MANMAD(MS)', 'MUMBAI',
       'NAGPUR', 'NEWASA(MS)', 'NIPHAD(MS)', 'PALAYAM(KER)', 'PATIALA(PB)',
       'PATNA', 'PHALTAN (MS)',

In [45]:
type(df.Market)

pandas.core.series.Series

In [46]:
import numpy as np

In [44]:
type(df.Market.values)

numpy.ndarray

## Downloading the Entire Month Wise Arrival Data

In [ ]:
AllTable = pd.read_html('MonthWiseMarketArrivals.html', header = 0,
                      attrs = {'id' : 'dnn_ctr974_MonthWiseMarketArrivals_GridView1'})

In [ ]:
AllTable[0].head()

In [ ]:
??pd.DataFrame.to_csv

In [ ]:
AllTable[0].columns

In [ ]:
# Change the column names to simpler ones
AllTable[0].columns = ['market', 'month', 'year', 'quantity', 'priceMin', 'priceMax', 'priceMod']

In [ ]:
AllTable[0].head()

In [ ]:
# Save the dataframe to a csv file
AllTable[0].to_csv('MonthWiseMarketArrivals.csv', index = False)